In [1]:
using BenchmarkTools

# Row/Column-major
Different programming languages use different memory layouts for arrays. For example matrices are stored in
row-major format in C/C++, Rust or Python; whereas they are stored in column-major format in Julia, Fortran and MATLAB.
Because of how data of these matrices is cached when we perform operations on them, the order in which we access the data
is important in terms of performance. In Julia we always want to iterate over the inner most index first, so we acces the arrays
memory contiguously and avoid cache misses

In [2]:
n = 256
A, B, C = rand(n,n), rand(n,n), rand(n,n)
function row_major!(A, B, C)
    for i in axes(A, 1), j in axes(A, 2)
        A[i, j] = B[i, j] * C[i, j]
    end
    return nothing
end
@btime row_major!($A, $B, $C);

  273.375 μs (0 allocations: 0 bytes)


In [3]:
function column_major!(A, B, C)
    for j in axes(A, 2), i in axes(A, 1)
        A[i, j] = B[i, j] * C[i, j]
    end
    return nothing
end
@btime column_major!($A, $B, $C);

  14.584 μs (0 allocations: 0 bytes)


Julia does biunds checking by default, but you can turn it off with the macro `@inbounds`,
usually resulting in faster code

In [4]:
function column_major_inbounds!(A, B, C)
    @inbounds for j in axes(A, 2), i in axes(A, 1)
        A[i, j] = B[i, j] * C[i, j]
    end
    return nothing
end
@btime column_major_inbounds!($A, $B, $C);

  14.542 μs (0 allocations: 0 bytes)


# Stack vs Heap allocation
These are the places in the memory where data is "stored". The stack is statically allocated and it is ordered.
This order allows the compiler to know exactly where things are, yielding very quick access. Like everything else that is referred as static
the size of variables (i.e. type and length) has to be known at compile time. On the other hand, the heap is dynamically allocated, and not necessarily is unordered, resulting in slower acces.
An excellent explanation of the difference between stack and heap can be found in [Rust's docs](https://web.mit.edu/rust-lang_v1.25/arch/amd64_ubuntu1404/share/doc/rust/html/book/first-edition/the-stack-and-the-heap.html).

In [5]:
allocate_heap() = [rand(); rand()]
@btime allocate_heap() # data size is a runtime parameter (~malloc)

  16.867 ns (1 allocation: 80 bytes)


2-element Vector{Float64}:
 0.6504407092201697
 0.9554583092089126

In [6]:
allocate_stack() = (rand(), rand()) # a tuple of two floats, everything is known at compile time
@btime allocate_stack()

  3.750 ns (0 allocations: 0 bytes)


(0.6736190105959982, 0.6820469885862294)

A very good performance-wise trick if we want to create non-allocating array-like objects is to use StaticArrays.jl

In [7]:
using StaticArrays
allocate_SA() = @SVector [rand(); rand()] # a tuple of two floats, everything is known at compile time
@btime allocate_SA()

  3.750 ns (0 allocations: 0 bytes)


2-element StaticArraysCore.SVector{2, Float64} with indices SOneTo(2):
 0.5553233997388317
 0.2683469755745207

# In-place vs out-of-place
It is common that you need to store the results of matrix operations in a new array. Creating the new destination array will obviously allocate, if you need to this operation just once, an out-of-place kernel will do just fine

In [8]:
function outofplace(A, B)
    C = similar(A)
    for i in axes(A, 1), j in axes(A, 2)
        C[i, j] = B[i, j] * C[i, j]
    end
    return C
end
@btime outofplace($A, $B)

  263.500 μs (2 allocations: 512.05 KiB)


256×256 Matrix{Float64}:
 0.0244012    7.58547e-5   7.46491e-10  …  1.81869e-5   0.0653567
 0.0020739    8.54529e-16  8.48845e-7      7.97624e-13  7.98994e-6
 2.68162e-12  3.06088e-17  6.56965e-8      1.73457e-17  0.430133
 1.03756e-8   0.56313      0.00152206      1.857e-14    1.40048e-6
 5.51295e-7   4.75049e-8   0.00770729      2.40125e-11  0.0919725
 6.27639e-34  8.7353e-22   6.78255e-5   …  2.30061e-12  9.85281e-14
 2.95727e-17  2.54384e-16  9.05094e-12     0.000604221  0.000291508
 1.83778e-17  4.5415e-15   1.40283e-12     0.0313915    0.240203
 1.48995e-5   0.918037     0.00147836      2.19671e-6   3.4345e-21
 1.23881e-38  3.36199e-6   5.22451e-12     1.61746e-7   1.55344e-7
 ⋮                                      ⋱               ⋮
 1.73668e-5   3.63267e-19  5.55119e-26     2.77805e-12  0.0425412
 2.02473e-21  1.2394e-32   0.000572216     8.17624e-8   1.59288e-8
 6.63307e-5   2.09905e-6   3.98218e-6      8.71846e-8   1.5168e-5
 0.0735741    3.47143e-6   1.3824e-17   …  6.55775e-

however, it is frequent you need to run the kernel several times, in which case you want to avoid allocating the destination array every time. In this case, you can use an in-place kernel, by pre-allocating the destinating array and just mutating it:

In [9]:
function inplace!(C, A, B)
    for i in axes(A, 1), j in axes(A, 2)
        C[i, j] = B[i, j] * C[i, j]
    end
    return nothing
end
@btime inplace!($C, $A, $B)

  272.167 μs (0 allocations: 0 bytes)


note that in the latter case we do no return anything, as Julia passes arguments by reference to the functions.
Also note the `!` at the end for the in-place function, this is a convention in Julia to denote that the function mutates its arguments (where the mutating arguments are the first ones).

# Array-like programming
# # Broadcasting
As in MATLAB, Julia allows to perform element-wise operations on arrays, this is called broadcasting. A dot before the target function indicates broadcasting

In [10]:
broadcast1(A, B, C) = A .+ B .+ C
@btime broadcast1($A, $B, $C)

  19.708 μs (2 allocations: 512.05 KiB)


256×256 Matrix{Float64}:
 1.69851    1.08498    0.874138   …  0.609179    0.708098  1.60865
 0.836012   0.366645   0.778152      0.387308    0.391743  0.88397
 0.714155   0.502699   0.693197      0.151027    0.361749  1.26867
 0.733409   1.73723    1.50097       1.61266     0.33491   0.726234
 0.802293   0.936798   0.992722      1.00761     0.736017  1.47277
 0.0678015  0.310121   1.20864    …  0.66941     0.791522  0.519745
 0.32115    0.39483    0.676584      1.02473     0.854787  1.24166
 0.291325   0.518057   0.444003      0.00195173  1.08073   1.15083
 1.20707    1.32184    1.4376        0.146819    0.961329  0.3307
 0.0652999  1.22418    0.489674      0.0878365   0.618127  0.957436
 ⋮                                ⋱                        ⋮
 0.998101   0.24874    0.132507      0.941759    0.561101  1.45079
 0.177123   0.0685122  1.56097       0.448772    0.76174   0.661713
 1.06105    1.17676    1.02949       0.0738419   0.869964  0.720023
 1.46343    0.942578   0.471918   …  0.

alternative you can use the macro `@.` to remove dot redundancy and apply broadcasting to all the operations

In [11]:
broadcast2(A, B, C) = @. A + B + C
@btime broadcast2($A, $B, $C)

  19.708 μs (2 allocations: 512.05 KiB)


256×256 Matrix{Float64}:
 1.69851    1.08498    0.874138   …  0.609179    0.708098  1.60865
 0.836012   0.366645   0.778152      0.387308    0.391743  0.88397
 0.714155   0.502699   0.693197      0.151027    0.361749  1.26867
 0.733409   1.73723    1.50097       1.61266     0.33491   0.726234
 0.802293   0.936798   0.992722      1.00761     0.736017  1.47277
 0.0678015  0.310121   1.20864    …  0.66941     0.791522  0.519745
 0.32115    0.39483    0.676584      1.02473     0.854787  1.24166
 0.291325   0.518057   0.444003      0.00195173  1.08073   1.15083
 1.20707    1.32184    1.4376        0.146819    0.961329  0.3307
 0.0652999  1.22418    0.489674      0.0878365   0.618127  0.957436
 ⋮                                ⋱                        ⋮
 0.998101   0.24874    0.132507      0.941759    0.561101  1.45079
 0.177123   0.0685122  1.56097       0.448772    0.76174   0.661713
 1.06105    1.17676    1.02949       0.0738419   0.869964  0.720023
 1.46343    0.942578   0.471918   …  0.

See that we are still allocating an output array. We can also do in-place operations with broadcasting by putting a dot before the equal symbol

In [12]:
function broadcasting3!(C, A, B)
    C .= A .+ B
    return nothing
end
@btime broadcasting3!($C, $A, $B);

  12.917 μs (0 allocations: 0 bytes)


# # Array slicing
We can use MATLAB syntax to slice our arrays

In [13]:
Aslice = A[:, 1];

However, this is creating a new array out of the slice of A (=allocating). If we want to avoid this, we can use the `@view` macro

In [14]:
@btime @. $C[:, 1] = $A[:, 1] + $B[:, 1]

  388.198 ns (2 allocations: 4.25 KiB)


256-element view(::Matrix{Float64}, :, 1) with eltype Float64:
 1.6985075371481333
 0.8360120260447305
 0.7141553882420562
 0.7334093492375442
 0.8022927112845272
 0.06780153724454925
 0.32115038479883007
 0.2913251116973369
 1.207071041922602
 0.06529989791887418
 ⋮
 0.9981006336701361
 0.1771226671547133
 1.0610539308661615
 1.4634349892443776
 0.4552349499243
 1.67197766810527
 0.580413430831008
 1.1639767251294675
 0.47668092370174275

But we can avoid allocations using the `@views` macro

In [15]:
@btime @views @. $C[:, 1] = $A[:, 1] + $B[:, 1]

  36.421 ns (0 allocations: 0 bytes)


256-element view(::Matrix{Float64}, :, 1) with eltype Float64:
 1.6985075371481333
 0.8360120260447305
 0.7141553882420562
 0.7334093492375442
 0.8022927112845272
 0.06780153724454925
 0.32115038479883007
 0.2913251116973369
 1.207071041922602
 0.06529989791887418
 ⋮
 0.9981006336701361
 0.1771226671547133
 1.0610539308661615
 1.4634349892443776
 0.4552349499243
 1.67197766810527
 0.580413430831008
 1.1639767251294675
 0.47668092370174275

Tip: loops are *fast*, so just loop. Looping often yields faster code, as you can do
more optimizations (e.g. loop fusion, caching, loop unrolling, SIMD, etc.), and more readible code

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*